<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/5_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# way 1

* 範例1

In [30]:
import pandas as pd
import numpy as np

def analyze_group(group):
    data = group['col5'].reset_index(drop=True)
    col6 = group['col6'].reset_index(drop=True)
    col7 = group['col7'].reset_index(drop=True)

    results = []  # 存储每个窗口是否满足条件的结果
    proportion_results = []  # 存储记录，用于计算比例
    # 计算滑动窗口
    for i in range(len(data) - 2):  # 窗口大小为3
        window = data.iloc[i:i + 3]
        count_ones = (window == 1).sum()

        # 检查是否满足条件
        condition_met = (i == 0 and count_ones >= 2) or (count_ones >= 2 and window.iloc[-1] == 1)
        results.append(condition_met)

        if condition_met:
            indices = window[window == 1].index.tolist()[::-1]
            # 对满足条件的窗口记录相关信息
            record = {
                'col1': group['col1'].iloc[0],
                'col2': group['col2'].iloc[0],
                'col3': group['col3'].iloc[0],
                'col4': group['col4'].iloc[0],
                'proportion': 1  # 后续会计算整体比例
            }
            # 为 col6 和 col7 添加最多三个值
            for j in range(3):
                if j < len(indices):
                    record[f'col6_{j+1}'] = col6.iloc[indices[j]]
                    record[f'col7_{j+1}'] = col7.iloc[indices[j]]
                else:
                    record[f'col6_{j+1}'] = np.nan
                    record[f'col7_{j+1}'] = np.nan
            proportion_results.append(record)

    if proportion_results:
        result_df = pd.DataFrame(proportion_results)
        overall_proportion = sum(results) / len(results) if results else 0
        result_df['proportion'] = overall_proportion
        return result_df
    else:
        # 如果没有记录满足条件，返回空 DataFrame
        return pd.DataFrame(columns=['col1', 'col2', 'col3', 'col4', 'proportion', 'col6_1', 'col7_1', 'col6_2', 'col7_2', 'col6_3', 'col7_3'])

# 示例数据
data = {
    'col1': ['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A'],
    'col2': ['aa', 'aa', 'aa', 'aa', 'aa', 'aa', 'aa', 'aa', 'aa', 'aa'],
    'col3': ['x', 'x', 'x', 'x', 'x', 'y', 'y', 'y', 'y', 'y'],
    'col4': ['z', 'z', 'z', 'z', 'z', 'w', 'w', 'w', 'w', 'w'],
    'col5': [0, 1, 1, 1, 0, 1, 0, 1, 1, 0],
    'col6': [10, 20, 30, 40, 50, 55, 60, 65, 70, 75],
    'col7': [200, 210, 220, 230, 240, 250, 260, 270, 280, 290]
}
df = pd.DataFrame(data)

# 应用函数并创建结果DataFrame
result_df = df.groupby(['col1', 'col2', 'col3', 'col4']).apply(analyze_group).reset_index(drop=True)
df


,col1,col2,col3,col4,col5,col6,col7
0,A,aa,x,z,0,10,200
1,A,aa,x,z,1,20,210
2,A,aa,x,z,1,30,220
3,A,aa,x,z,1,40,230
4,A,aa,x,z,0,50,240
5,A,aa,y,w,1,55,250
6,A,aa,y,w,0,60,260
7,A,aa,y,w,1,65,270
8,A,aa,y,w,1,70,280
9,A,aa,y,w,0,75,290


In [31]:
df.groupby(['col1', 'col2', 'col3', 'col4']).size().reset_index()

,col1,col2,col3,col4,0
0,A,aa,x,z,5
1,A,aa,y,w,5


In [32]:
result_df

,col1,col2,col3,col4,proportion,col6_1,col7_1,col6_2,col7_2,col6_3,col7_3
0,A,aa,x,z,0.666667,30,220,20,210,NaN,NaN
1,A,aa,x,z,0.666667,40,230,30,220,20.0,210.0
2,A,aa,y,w,0.666667,65,270,55,250,NaN,NaN
3,A,aa,y,w,0.666667,70,280,65,270,NaN,NaN


* 範例2

In [25]:
data = {
    'col1': ['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B'],
    'col2': ['aa', 'aa', 'aa', 'aa', 'aa', 'aa', 'aa', 'aa', 'aa', 'aa', 'bb', 'bb', 'bb', 'bb', 'bb', 'bb', 'bb', 'bb', 'bb', 'bb', 'bb'],
    'col3': ['x', 'x', 'x', 'x', 'x', 'y', 'y', 'y', 'y', 'y', 'x', 'x', 'x', 'y', 'y', 'y', 'y', 'y', 'y', 'y', 'y'],
    'col4': ['z', 'z', 'z', 'z', 'z', 'w', 'w', 'w', 'w', 'w', 'z', 'z', 'z', 'z', 'z', 'w', 'w', 'w', 'w', 'w', 'z'],
    'col5': [0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0],
    'col6': [10, 20, 30, 40, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 105, 110, 115, 120, 125, 130],
    'col7': [200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380, 390, 400]
}
df = pd.DataFrame(data)
df

,col1,col2,col3,col4,col5,col6,col7
0,A,aa,x,z,0,10,200
1,A,aa,x,z,1,20,210
2,A,aa,x,z,1,30,220
3,A,aa,x,z,1,40,230
4,A,aa,x,z,0,50,240
5,A,aa,y,w,1,55,250
6,A,aa,y,w,0,60,260
7,A,aa,y,w,1,65,270
8,A,aa,y,w,1,70,280
9,A,aa,y,w,0,75,290


In [28]:
df.groupby(['col1', 'col2', 'col3', 'col4']).size().reset_index()

,col1,col2,col3,col4,0
0,A,aa,x,z,5
1,A,aa,y,w,5
2,B,bb,x,z,3
3,B,bb,y,w,5
4,B,bb,y,z,3


In [26]:
df.groupby(['col1', 'col2', 'col3', 'col4']).apply(analyze_group).reset_index(drop=True)

,col1,col2,col3,col4,proportion,col6_1,col7_1,col6_2,col7_2,col6_3,col7_3
0,A,aa,x,z,0.666667,30,220,20,210,NaN,NaN
1,A,aa,x,z,0.666667,40,230,30,220,20.0,210.0
2,A,aa,y,w,0.666667,65,270,55,250,NaN,NaN
3,A,aa,y,w,0.666667,70,280,65,270,NaN,NaN
4,B,bb,x,z,1.000000,90,320,85,310,NaN,NaN
5,B,bb,y,w,0.666667,115,370,105,350,NaN,NaN
6,B,bb,y,w,0.666667,120,380,115,370,NaN,NaN


# way 2

In [34]:
import pandas as pd
import numpy as np

def analyze_group(group):
    data = group['col5'].reset_index(drop=True)
    col6 = group['col6'].reset_index(drop=True)
    col7 = group['col7'].reset_index(drop=True)

    results = []
    proportion_results = []  # 存储记录，用于计算比例
    # 计算滑动窗口
    for i in range(len(data) - 2):  # 窗口大小为3
        window = data.iloc[i:i + 3]
        count_ones = (window == 1).sum()

        # 检查是否满足条件
        condition_met = (i == 0 and count_ones >= 2) or (count_ones >= 2 and window.iloc[-1] == 1)
        results.append(condition_met)

        if condition_met:
            indices = window[window == 1].index.tolist()
            # 收集满足条件的窗口中的col6, col7值（倒序）
            reversed_indices = indices[::-1]  # 将索引倒序
            col6_values = [col6.iloc[idx] for idx in reversed_indices]
            col7_values = [col7.iloc[idx] for idx in reversed_indices]

            record = {
                'col1': group['col1'].iloc[0],
                'col2': group['col2'].iloc[0],
                'col3': group['col3'].iloc[0],
                'col4': group['col4'].iloc[0],
                'proportion': 1  # 后续会统一计算整体比例
            }

            # 将倒序的col6和col7添加到记录中
            for j in range(3):
                if j < len(col6_values):
                    record[f'col6_{j+1}'] = col6_values[j]
                    record[f'col7_{j+1}'] = col7_values[j]
                else:
                    record[f'col6_{j+1}'] = np.nan
                    record[f'col7_{j+1}'] = np.nan
            proportion_results.append(record)

    if proportion_results:
        result_df = pd.DataFrame(proportion_results)
        overall_proportion = sum(results) / len(results) if results else 0
        result_df['proportion'] = overall_proportion
        return result_df
    else:
        # 如果没有记录满足条件，返回空 DataFrame
        return pd.DataFrame(columns=['col1', 'col2', 'col3', 'col4', 'proportion', 'col6_1', 'col7_1', 'col6_2', 'col7_2', 'col6_3', 'col7_3'])

# 应用函数并创建结果DataFrame
result_df = df.groupby(['col1', 'col2', 'col3', 'col4']).apply(analyze_group).reset_index(drop=True)
result_df


,col1,col2,col3,col4,proportion,col6_1,col7_1,col6_2,col7_2,col6_3,col7_3
0,A,aa,x,z,0.666667,30,220,20,210,NaN,NaN
1,A,aa,x,z,0.666667,40,230,30,220,20.0,210.0
2,A,aa,y,w,0.666667,65,270,55,250,NaN,NaN
3,A,aa,y,w,0.666667,70,280,65,270,NaN,NaN
